# Technical Report  
Darrell Cenido and Jon Larson  
CPSC 322, Fall 2025

## Introduction
The dataset we are using is a [Spotify track dataset](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-600k-tracks) from Kaggle that contains attributes such as popularity (which we are planning on classifying), danceability, duration_ms, artists, explicit, energy, release_date, etc.  

[We are planning on using the attributes mentioned above to help classify how popular a track is.]

> add a brief description of findings (e.g., what classifier approach performed the best)

## Data Analysis

> Data Analysis: This section must provide details about the dataset. You must include:
> 1. Information about the dataset itself, e.g., the attributes and attribute types, the number of instances, and the attribute being used as the label.
> 1. Relevant summary statistics about the dataset.
> 1. Data visualizations highlighting important/interesting aspects of your dataset. Visualizations may include frequency distributions, comparisons of attributes (scatterplot, multiple frequency diagrams), box and whisker plots, etc. The goal is not to include all possible diagrams, but instead to select and highlight diagrams that provide insight about the dataset itself.
> 1. Note that this section must describe the above (in paragraph form) and not just provide diagrams and statistics. Also, each figure included must have a figure caption (Figure number and textual description) that is referenced from the text (e.g., “Figure 2 shows a frequency diagram for ...”).

## Classification Results

 > Classification Results: This section should describe the classification approach you developed and its performance. Explain what techniques you used, briefly how you designed and implemented the classifiers, how you evaluated your classifiers’ predictive ability, and how well the classifiers performed. Thoroughly describe how you evaluated performance, the comparison results, and which classifier is “best”.

## Conclusion

 > Conclusion: Provide a brief conclusion of your project, including a short summary of the dataset you used (and any of its inherent challenges for classification), the classification approach you developed, your classifiers’ performance, and any ideas you have on ways to improve its performance.

## Acknowledgments

> Acknowledgments: This is where you should cite your sources, including any data, code, or materials that are outside of the scope of CPSC 322 (including previous course projects) that you used. As per the course syllabus, you also need to acknowledge any use of AI.